##### second improvement
* previous_click_time
    * ip, channel 
* next_click_time
    * app, device, channel
    * device, hour
    * ip, app, device, os, channel
    * ip, device, os
* cumcount
    * ip, device, os, [app]
    * ip, device, os, app, [channel]

##### third improvement
* mean: 
    * ip, app, channel, [hour] 
* var:
    * ip, app, day, [hour]
    * ip, device, day, [hour]
    * ip, os, day, [hour]

In [2]:
import pandas as pd
import time
import numpy as np
import gc
import matplotlib.pyplot as plt
import os
import pickle
from datetime import datetime

In [2]:
dtypes = {
            'ip'            : 'uint32',
            'app'           : 'uint16',
            'device'        : 'uint8',
            'os'            : 'uint16',
            'channel'       : 'uint16',
            'is_attributed' : 'uint8',
            'click_id'      : 'uint32',
            'day'           : 'unit16',
            'hour'          : 'unit16',
            'minutes'       : 'unit16',
            'seconds'       : 'unit16'
            } 


In [3]:
cols =  ['app', 'app_count', 'appchannel_count', 'appdevice_count',
       'appos_count', 'channel', 'channel_count', 'day', 'device',
       'dido_first', 'didoa_first', 'didoa_pct', 'hour', 'iac_count',
       'ida_count', 'ido_count', 'ido_pct', 'ioc_count', 'ip', 'ip_count',
       'ipapp_count', 'ipdevice_count', 'ipos_count',
        'napp_byido', 'napp_byip', 'nchannel_byip',
       'ndevice_byip', 'nhour_byipday', 'nos_byipapp', 'os', 'os_count', 'set']

In [2]:
train = pd.read_csv('/Volumes/My Passport for Mac/full_data_0702.csv', index_col = 0)

/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [9]:
train.click_time = pd.to_datetime(train.click_time, errors = 'ignore')

In [6]:
train.hour.unique()

array([14, 15,  4,  5,  6,  9, 10, 11, 12, 13])

In [9]:
train[(train.day == 6) & (train.ip == 209663) & (train.device == 1) & (train.os == 13)]

,ip,app,device,os,channel,click_time,day,weekday,hour,minutes,...,ipos_count,appdevice_count,appchannel_count,appos_count,ida_count,ido_count,ioc_count,iac_count,ido_pct,didoa_pct
27,209663,3,1,13,379,2017-11-06 14:48:55,6,0,14,48,...,16050,19293035,646698,4443880,9681,13634,119,191,0,0
283,209663,3,1,13,379,2017-11-06 15:49:43,6,0,15,49,...,16050,19293035,646698,4443880,9681,13634,119,191,3648,3648


### previous_click time

In [7]:
def pre_click_time(groupby_col):
    previous_clicktime = train2[groupby_col + ['click_time']].groupby(groupby_col)['click_time'].shift(+1)
    diff = (train2.click_time - previous_clicktime).dt.seconds.astype('float32') 
    return diff

In [10]:
ipchannel_pct = pre_click_time(['ip', 'channel'])

In [11]:
ipchannel_pct.shape

(118123128,)

In [47]:
ipchannel_pct.to_csv('/Users/mengruiyin/Dropbox/project3/project3_coding/new feature/full_data_feature/previous_click.csv')

### next-click time

In [3]:
fmt = '%Y-%m-%d %H:%M:%S'

In [5]:
current_time = pd.DataFrame(train['click_time'])

In [5]:
dido_nct = train[['day', 'ip', 'device', 'os', 'click_time']].groupby(['day', 'ip', 'device', 'os'])['click_time'].shift(-1)

In [8]:
dido_nct = np.where(dido_nct.isnull(), current_time['click_time'], dido_nct)

In [9]:
dido_nct = pd.DataFrame(dido_nct)
dido_nct.columns = ['next_click_time']

In [10]:
dido_nct.head()

,next_click_time
0,2017-11-06 14:32:21
1,2017-11-06 14:33:34
2,2017-11-06 14:34:12
3,2017-11-06 14:34:52
4,2017-11-06 14:35:08


In [11]:
dido_nct = dido_nct['next_click_time'].apply(lambda x: datetime.strptime(str(x), fmt))

In [12]:
current_time = current_time['click_time'].apply(lambda x: datetime.strptime(str(x), fmt))

In [13]:
train.head()

,ip,app,device,os,channel,click_time,day,weekday,hour,minutes,...,ipos_count,appdevice_count,appchannel_count,appos_count,ida_count,ido_count,ioc_count,iac_count,ido_pct,didoa_pct
0,83230,3,1,13,379,2017-11-06 14:32:21,6,0,14,32,...,3500,19293035,646698,4443880,2888,3473,46,101,0,0
1,17357,3,1,19,379,2017-11-06 14:33:34,6,0,14,33,...,4107,19293035,646698,4793727,2725,4105,65,101,0,0
2,35810,3,1,13,379,2017-11-06 14:34:12,6,0,14,34,...,1309,19293035,646698,4443880,1208,1084,16,28,0,0
3,45745,14,1,13,478,2017-11-06 14:34:52,6,0,14,34,...,23845,6169877,11768,1366829,6425,20948,89,35,0,0
4,161007,3,1,13,379,2017-11-06 14:35:08,6,0,14,35,...,134,19293035,646698,4443880,82,134,5,3,0,0


In [14]:
train['dido_nct'] = dido_nct - current_time

In [15]:
train['dido_nct']  = train['dido_nct'].apply(lambda x: x.seconds)

In [17]:
dido_nct_df = pd.DataFrame(train['dido_nct'])

In [21]:
dido_nct_df.to_csv('/Users/mengruiyin/Dropbox/project3/project3_coding/dido_nct_df.csv')

In [4]:
dido_nct_df = pd.read_csv('/Users/mengruiyin/Dropbox/project3/project3_coding/dido_nct_df.csv', index_col = 0)

FileNotFoundError: File b'/Users/mengruiyin/Dropbox/project3/project3_coding/dido_nct_df.csv' does not exist

In [6]:
dido_nct_df.head()

,dido_nct
0,0
1,0
2,0
3,0
4,0


In [7]:
train['dido_nct'] = dido_nct_df['dido_nct'].values

In [8]:
train['dido_last'] = np.where(train['dido_nct'] == 0, 1, 0)

In [9]:
gc.collect()

21

In [11]:
dido_last_df = pd.DataFrame(train['dido_last'])

In [12]:
dido_last_df.to_csv('/Users/mengruiyin/Dropbox/project3/project3_coding/new feature/full_data_feature/dido_last.csv')

In [13]:
gc.collect()

0

In [21]:
def next_click_time(groupby_col):
    next_clicktime = train[groupby_col + ['click_time']].groupby(groupby_col)['click_time'].shift(-1)
    diff = (next_clicktime - train.click_time).dt.seconds.astype('float32') 
    return diff

In [22]:
gc.collect()

96

In [23]:
adc_nct = next_click_time(['app', 'device', 'channel'])

In [28]:
dh_nct = next_click_time(['device', 'hour'])

In [30]:
# dido_nct = next_click_time(['day', 'ip', 'device', 'os'])

In [34]:
iadoc_nct = next_click_time(['ip', 'app', 'device', 'os', 'channel'])

In [33]:
ido_nct = next_click_time(['ip', 'device', 'os'])

In [35]:
next_click_add = pd.DataFrame({'adc_nct': adc_nct, 'dh_nct': dh_nct, 'iadoc_nct': iadoc_nct, 'ido_nct': ido_nct})

In [40]:
next_click_add = next_click_add.fillna(0)

In [42]:
next_click_add.head(100)

,adc_nct,dh_nct,iadoc_nct,ido_nct
0,73.0,73.0,49497.0,48503.0
1,38.0,38.0,51002.0,48589.0
2,56.0,40.0,75067.0,48598.0
3,599.0,16.0,50206.0,48323.0
4,78.0,78.0,50757.0,50481.0
5,78.0,78.0,53965.0,53539.0
6,15.0,15.0,0.0,82474.0
7,11.0,11.0,75849.0,49003.0
8,17.0,13.0,0.0,50044.0
9,996.0,4.0,0.0,51127.0


In [44]:
next_click_add.to_csv('/Users/mengruiyin/Dropbox/project3/project3_coding/new feature/full_data_feature/next_click.csv')

In [ ]:
# didoa_nct = train[['day', 'ip', 'device', 'os', 'app', 'click_time']].groupby(['day', 'ip', 'device', 'os', 'app'])['click_time'].shift(-1)

In [48]:
gc.collect()

215

### cumcount

In [3]:
def cum_count(groupby_col, cumcount_col):
    cumcount_val = train2[groupby_col + [cumcount_col]].groupby(groupby_col)[cumcount_col].cumcount()
    return cumcount_val

In [4]:
ido_app = cum_count(['ip', 'device', 'os'], 'app')

KeyboardInterrupt: 

In [53]:
ido_app.head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
dtype: int64

In [55]:
idoa_channel = cum_count(['ip', 'device', 'os', 'app'], 'channel')

In [57]:
ip_os = cum_count(['ip'], 'os')

In [58]:
cum_add = pd.DataFrame({'ido_app_cum': ido_app, 'idoa_channel_cum': idoa_channel, 'ip_os_cum': ip_os})

In [59]:
cum_add.shape

(118123128, 3)

In [62]:
cum_add.tail(100)

,ido_app_cum,idoa_channel_cum,ip_os_cum
118123028,5783,378,27733
118123029,11992,2296,503513
118123030,413,14,2023
118123031,11052,939,65466
118123032,6118,727,162951
118123033,844,432,41595
118123034,20,6,73
118123035,4207,88,260101
118123036,240,47,11539
118123037,4566,64,21214


In [63]:
cum_add.to_csv('/Users/mengruiyin/Dropbox/project3/project3_coding/new feature/full_data_feature/cumcount_feature.csv')

In [5]:
next_click_fea = pd.read_csv('/Users/mengruiyin/Dropbox/project3/project3_coding/new feature/full_data_feature/next_click.csv', index_col = 0)

/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
next_click_fea.head()

,adc_nct,dh_nct,iadoc_nct,ido_nct
0,73.0,73.0,49497.0,48503.0
1,38.0,38.0,51002.0,48589.0
2,56.0,40.0,75067.0,48598.0
3,599.0,16.0,50206.0,48323.0
4,78.0,78.0,50757.0,50481.0


### mean

In [4]:
def mean_value(groupby_col, mean_col):
    mean_value = train2[groupby_col + [mean_col]].groupby(groupby_col)[mean_col].transform(np.mean)
    return mean_value

In [5]:
train2['mhour_iac'] = mean_value(['ip', 'app', 'channel'], 'hour')

In [6]:
train2['mhour_iac'].isnull().sum()

0

### var

In [7]:
def var_value(groupby_col, var_col):
    var_value = train2[groupby_col + [var_col]].groupby(groupby_col)[var_col].var()
    return var_value

In [8]:
vhour_iad = var_value(['ip', 'app', 'day'], 'hour')

In [9]:
vhour_iad = pd.DataFrame(vhour_iad).reset_index()

In [10]:
vhour_iad.rename(columns={'hour': 'vhour_iad'}, inplace = True)

In [11]:
vhour_idd = pd.DataFrame(var_value(['ip', 'device', 'day'], 'hour')).reset_index()

In [12]:
vhour_idd.rename(columns={'hour': 'vhour_idd'}, inplace = True)

In [13]:
vhour_iod = pd.DataFrame(var_value(['ip', 'os', 'day'], 'hour')).reset_index()

In [14]:
vhour_iod.rename(columns={'hour': 'vhour_iod'}, inplace = True)

In [15]:
vhour_iad.fillna(0, inplace = True)
vhour_idd.fillna(0, inplace = True)
vhour_iod.fillna(0, inplace = True)

In [16]:
vhour_iad['vhour_iad'].isnull().sum()

0

In [17]:
train2[(train2.ip == 1) & (train2.device == 1) & (train2.day == 10)]['hour'].var()

nan

### training set features and testing set features

##### next click feature

In [16]:
train2 = train[['ip', 'app', 'device', 'os', 'channel', 'click_time', 'day', 'hour']]

In [17]:
train2['adc_nct'] = next_click_fea['adc_nct']

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
train2['dh_nct'] = next_click_fea['dh_nct']

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
train2['ladoc_nct'] = next_click_fea['iadoc_nct']

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
train2['ido_nct'] = next_click_fea['ido_nct']

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
train2.to_csv('temp_train2.csv')

In [2]:
train2 = pd.read_csv('/Volumes/My Passport for Mac/temp_train2.csv', index_col = 0)

/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
train2.head()

,ip,app,device,os,channel,click_time,day,hour,adc_nct,dh_nct,ladoc_nct,ido_nct
0,83230,3,1,13,379,2017-11-06 14:32:21,6,14,73.0,73.0,49497.0,48503.0
1,17357,3,1,19,379,2017-11-06 14:33:34,6,14,38.0,38.0,51002.0,48589.0
2,35810,3,1,13,379,2017-11-06 14:34:12,6,14,56.0,40.0,75067.0,48598.0
3,45745,14,1,13,478,2017-11-06 14:34:52,6,14,599.0,16.0,50206.0,48323.0
4,161007,3,1,13,379,2017-11-06 14:35:08,6,14,78.0,78.0,50757.0,50481.0


In [4]:
not12 = train2[train2.hour != 12]

In [9]:
training_8 = not12[(not12.day == 8) | (not12.day == 9)]

In [13]:
training_8.shape

(61035363, 12)

In [8]:
not12.shape

(108273763, 12)

In [11]:
training_8.head()

,ip,app,device,os,channel,click_time,day,hour,adc_nct,dh_nct,ladoc_nct,ido_nct
31448022,58991,12,1,26,259,2017-11-08 04:00:00,8,4,0.0,0.0,0.0,1.0
31448023,184626,18,1,19,121,2017-11-08 04:00:00,8,4,0.0,0.0,322.0,0.0
31448024,151871,3,1,9,280,2017-11-08 04:00:00,8,4,0.0,0.0,0.0,29659.0
31448025,4180,3,1,19,424,2017-11-08 04:00:00,8,4,0.0,0.0,22000.0,1.0
31448026,231270,9,1,15,466,2017-11-08 04:00:00,8,4,0.0,0.0,0.0,7.0


In [12]:
training_8.to_csv('/Users/mengruiyin/Dropbox/project3/project3_coding/new feature/training_set_feature/nct_new_feature.csv')

In [14]:
testing_nct_new = not12[not12.day == 10]

In [15]:
testing_nct_new.shape

(18790469, 12)

In [16]:
testing_nct_new.to_csv('/Users/mengruiyin/Dropbox/project3/project3_coding/new feature/testing_set_feature/nct_new_feature.csv')

(18790469, 12)

##### pre click_time & cumcount

In [3]:
# previous_click = pd.read_csv('/Users/mengruiyin/Dropbox/project3/project3_coding/new feature/full_data_feature/previous_click.csv', index_col = 0)

/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [12]:
cum_count = pd.read_csv('/Users/mengruiyin/Dropbox/project3/project3_coding/new feature/full_data_feature/cumcount_feature.csv', index_col = 0)

/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [14]:
cum_count.shape

(118123128, 3)

In [13]:
ipchannel_pct.shape

(118123128,)

In [8]:
train.shape

(118123128, 28)

In [16]:
train2 = pd.read_csv('temp_train2.csv', index_col = 0)

/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [17]:
gc.collect()

14

In [15]:
train2['ipchannel_pct'] = ipchannel_pct.values

In [16]:
train2['ido_app_cum'] = cum_count['ido_app_cum'].values

In [17]:
train2['idoa_channel_cum'] = cum_count['idoa_channel_cum'].values

In [18]:
train2['ip_os_cum'] = cum_count['ip_os_cum'].values

In [20]:
train2 = train2.fillna(0)

In [21]:
gc.collect()

0

In [24]:
train2.to_csv('temp_train2_2.csv')

In [25]:
gc.collect()

0

In [26]:
not12 = train2[train2.hour != 12]

In [27]:
gc.collect()

0

In [28]:
training_add2 = not12[(not12.day == 8) | (not12.day == 9)]

In [29]:
training_add2.head()

,ip,app,device,os,channel,click_time,day,hour,adc_nct,dh_nct,ladoc_nct,ido_nct,ipchannel_pct,ido_app_cum,idoa_channel_cum,ip_os_cum
31448022,58991,12,1,26,259,2017-11-08 04:00:00,8,4,0.0,0.0,0.0,1.0,0.0,0,0,0
31448023,184626,18,1,19,121,2017-11-08 04:00:00,8,4,0.0,0.0,322.0,0.0,56308.0,9,0,97
31448024,151871,3,1,9,280,2017-11-08 04:00:00,8,4,0.0,0.0,0.0,29659.0,50236.0,1,1,637
31448025,4180,3,1,19,424,2017-11-08 04:00:00,8,4,0.0,0.0,22000.0,1.0,49642.0,73,18,529
31448026,231270,9,1,15,466,2017-11-08 04:00:00,8,4,0.0,0.0,0.0,7.0,0.0,0,0,0


In [30]:
training_add2.shape

(61035363, 16)

In [31]:
testing_add2 = not12[not12.day == 10]

In [32]:
testing_add2.shape

(18790469, 16)

In [33]:
training_add2.to_csv('/Users/mengruiyin/Dropbox/project3/project3_coding/new feature/training_set_feature/new_feature2.csv')

In [34]:
testing_add2.to_csv('/Users/mengruiyin/Dropbox/project3/project3_coding/new feature/testing_set_feature/new_feature2.csv')

##### var

In [57]:
vhour_iod.head()

,ip,os,day,vhour_iod
0,0,0,10,0.0
1,1,2,9,0.0
2,1,4,9,2.0
3,1,7,10,0.0
4,1,27,9,0.0


In [18]:
train2 = pd.merge(train2, vhour_iad, on = ['ip', 'app', 'day'], how = 'left')

In [20]:
train2 = pd.merge(train2, vhour_idd, on = ['ip', 'device', 'day'], how = 'left' )

In [23]:
train2 = pd.merge(train2, vhour_iod, on = ['ip', 'os', 'day'], how = 'left')

In [21]:
gc.collect()

21

In [22]:
train2.head()

,ip,app,device,os,channel,click_time,day,hour,adc_nct,dh_nct,ladoc_nct,ido_nct,mhour_iac,vhour_iad,vhour_idd
0,83230,3,1,13,379,2017-11-06 14:32:21,6,14,73.0,73.0,49497.0,48503.0,8.683168,0.0,0.0
1,17357,3,1,19,379,2017-11-06 14:33:34,6,14,38.0,38.0,51002.0,48589.0,8.811881,0.5,0.5
2,35810,3,1,13,379,2017-11-06 14:34:12,6,14,56.0,40.0,75067.0,48598.0,10.821429,0.0,0.0
3,45745,14,1,13,478,2017-11-06 14:34:52,6,14,599.0,16.0,50206.0,48323.0,10.257143,0.0,0.0
4,161007,3,1,13,379,2017-11-06 14:35:08,6,14,78.0,78.0,50757.0,50481.0,7.666667,0.0,0.0


In [25]:
train2.to_csv('train_addmv.csv')

In [28]:
train2.shape

(118123128, 16)

In [3]:
train2 = pd.read_csv('train_addmv.csv', index_col = 0)

/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
not12 = train2[train2.hour != 12]

In [5]:
training_set = not12[(not12.day == 8) | (not12.day == 9)]

In [7]:
training_set.to_csv('training_set_addmv.csv')

In [8]:
training_set.head()

,ip,app,device,os,channel,click_time,day,hour,adc_nct,dh_nct,ladoc_nct,ido_nct,mhour_iac,vhour_iad,vhour_idd,vhour_iod
31448022,58991,12,1,26,259,2017-11-08 04:00:00,8,4,0.0,0.0,0.0,1.0,9.454545,12.782313,13.230542,0.000000
31448023,184626,18,1,19,121,2017-11-08 04:00:00,8,4,0.0,0.0,322.0,0.0,6.166667,12.992424,10.537760,0.064368
31448024,151871,3,1,9,280,2017-11-08 04:00:00,8,4,0.0,0.0,0.0,29659.0,7.808824,13.356897,12.538675,0.000000
31448025,4180,3,1,19,424,2017-11-08 04:00:00,8,4,0.0,0.0,22000.0,1.0,8.952381,10.582463,10.232353,7.623490
31448026,231270,9,1,15,466,2017-11-08 04:00:00,8,4,0.0,0.0,0.0,7.0,12.200000,8.493103,16.847467,0.000000


In [9]:
test_set = not12[not12.day == 10]

In [11]:
test_set.to_csv('test_set_addmv.csv')